In [ ]:
!pip3 install stellargraph
# # !pip3 install pandas
# !pip3 install sklearn

  Using cached stellargraph-1.2.1-py3-none-any.whl (435 kB)
  Using cached gensim-3.8.3-cp37-cp37m-manylinux1_x86_64.whl (24.2 MB)
     |████████████████████████████████| 11.6 MB 1.6 MB/s eta 0:00:01
  Using cached networkx-2.5-py3-none-any.whl (1.6 MB)
     |████████████████████████████████| 6.8 MB 34.2 MB/s eta 0:00:01
     |████████████████████████████████| 9.5 MB 40.4 MB/s eta 0:00:01
     |████████████████████████████████| 113 kB 39.3 MB/s eta 0:00:01
  Using cached cycler-0.10.0-py2.py3-none-any.whl (6.5 kB)
     |████████████████████████████████| 1.1 MB 42.9 MB/s eta 0:00:01
     |████████████████████████████████| 2.2 MB 34.4 MB/s eta 0:00:01
  Using cached pyparsing-2.4.7-py2.py3-none-any.whl (67 kB)
  Using cached python_dateutil-2.8.1-py2.py3-none-any.whl (227 kB)
  Using cached decorator-4.4.2-py2.py3-none-any.whl (9.2 kB)
  Using cached threadpoolctl-2.1.0-py3-none-any.whl (12 kB)
  Using cached joblib-0.17.0-py3-none-any.whl (301 kB)
     |████████████████████████████████|

  Created wheel for smart-open: filename=smart_open-3.0.0-py3-none-any.whl size=107095 sha256=de1bb52637178ab2961eea36f7b2f3f75b7675ca05b14e609dd0718dfca4ea53
  Stored in directory: /home/anoopkumar/.cache/pip/wheels/83/a6/12/bf3c1a667bde4251be5b7a3368b2d604c9af2105b5c1cb1870
Successfully built smart-open


In [7]:
import pickle
import numpy as np
import pandas as pd
import sklearn
import tensorflow as tf
from copy import deepcopy
from stellargraph import StellarGraph
import time

ModuleNotFoundError: No module named 'stellargraph'

In [2]:
!pwd

/home/anoopkumar


In [3]:
all_sub_adjMat = np.load("vibha_preprop_adni/correlation_matrices.npy")
sub_names = np.load("vibha_preprop_adni/filenamelist.npy", allow_pickle= True)
print(len(all_sub_adjMat), len(sub_names))

152 152


In [4]:
subnames = [str(i)[5:15] for i in sub_names]
print(len(subnames), subnames[:5])

152 ['053_S_0919', '100_S_4511', '100_S_4469', '130_S_4468', '100_S_4884']


In [5]:
with open('/home/anoopkumar/adni/anoop/bck/filelist.txt') as f:
    content = f.readlines()
content = [x.strip() for x in content]
content.pop(-1)
print(len(content))

172


In [6]:
df = pd.read_csv('/home/anoopkumar/adni/anoop/bck/xlabel.csv') # can also index sheet by name or fetch all sheets
label = df['Group'].tolist()
subject = df['Subject'].tolist()
print(len(label),label[:5], subject[:5])

172 ['CN', 'CN', 'CN', 'MCI', 'MCI'] ['002_S_0295', '002_S_0413', '002_S_0685', '002_S_0729', '002_S_1155']


In [7]:
adni90_feature = []
adni90_labels = []
adni90_adjmat = []

for x,i in enumerate(subnames):
    ind = subject.index(i)
    adni90_labels.append(label[ind])
    adni90_feature.append(all_sub_adjMat[x])

for i in adni90_feature:
    temp = np.zeros((90,90))
    for x in range(90):
        for y in range(90):
            if i[x][y] > 0.15:
                temp[x][y] = 1
    adni90_adjmat.append(list(temp))
            
    
print(len(adni90_labels), len(adni90_adjmat), len(adni90_feature))

152 152 152


In [9]:
############################# gcn code ###############################################################

In [8]:
all_sg = [[] for i in range(152)]

for count, sub in enumerate(adni90_adjmat):
    np.fill_diagonal(np.array(sub), np.nan)
    df = pd.DataFrame(sub)
    df = df.stack().reset_index()
    df.columns = ['a','b','c']
    isedge = df['c'] == 1
    dfn = df[isedge]
    pairs = list(zip(dfn['a'], dfn['b']))
    for i in pairs:
            a, b = i
            if (b,a) not in all_sg[count]:
                all_sg[count].append(i)

In [9]:
print(len(all_sg))
print(len(all_sg[0]))

152
1987


In [10]:
all_sg_lr = [[] for _ in range(152)]

for i,x in enumerate(all_sg):
    all_sg_lr[i] = [[],[]]
    for pair in x:
        a, b = pair
        all_sg_lr[i][0].append(a)
        all_sg_lr[i][1].append(b)
        

In [24]:
# for featue

all_sub_fea = []
for i in adni90_feature:
    fead = pd.DataFrame(i)
    all_sub_fea.append(fead)

In [30]:
graphs = []

for i,one_sglr in enumerate(all_sg_lr):
    dic = {"source": one_sglr[0],"target":one_sglr[1]}
    one = pd.DataFrame(dic)
    graphs.append(StellarGraph(all_sub_fea[i], edge = one))

In [31]:
print(graphs[1].info())

StellarGraph: Undirected multigraph
 Nodes: 90, Edges: 2170

 Node types:
  default: [90]
    Edge types: default-default->default

 Edge types:
    default-default->default: [2170]
        Attributes: {'weight'}



In [ ]:
summary = pd.DataFrame(
    [(g.number_of_nodes(), g.number_of_edges()) for g in graphs],
    columns=["nodes", "edges"],
)
summary.describe().round(1)

In [ ]:

################### EMCI Vs LMCI  #############################
graph_labels = []
rem_ad = []
exp1 = []
for c,i in enumerate(adni90_labels):
    if i == 'EMCI':
        graph_labels.append(1)
        exp1.append(graphs[c])
    elif i == 'LMCI':
        graph_labels.append(-1)
        exp1.append(graphs[c])
    else:
        rem_ad.append(c)
   

################## AD Vs NC ################################
# graph_labels = []
# rem_ad = []
# exp1 = []
# for c,i in enumerate(all_sub_label):
#     if i == 'AD':
#         graph_labels.append(1)
#         exp1.append(graphs[c])
#     elif i == 'CN':
#         graph_labels.append(-1)
#         exp1.append(graphs[c])
#     else:
#         rem_ad.append(c)


################  AD Vs MCI ###############################
# graph_labels = []
# rem_ad = []
# exp1 = []
# for c,i in enumerate(all_sub_label):
#     if i == 'EMCI' or i == 'LMCI' or i == 'MCI':
#         graph_labels.append(1)
#         exp1.append(graphs[c])
#     elif i == 'AD':
#         graph_labels.append(-1)
#         exp1.append(graphs[c])
#     else:
#         rem_ad.append(c)
   
   


########### MCI Vs NC ##########################
# graph_labels = []
# rem_ad = []
# exp1 = []
# for c,i in enumerate(all_sub_label):
#     if i == 'EMCI' or i == 'LMCI' or i == 'MCI':
#         graph_labels.append(1)
#         exp1.append(graphs[c])
#     elif i == 'CN':
#         graph_labels.append(-1)
#         exp1.append(graphs[c])
#     else:
#         rem_ad.append(c)
    

In [ ]:
print(len(exp1), len (graph_labels))

In [ ]:
pd.Series(graph_labels).value_counts().to_frame()

In [15]:
graph_labels = pd.get_dummies(graph_labels, drop_first=True)
# print(graph_labels)

NameError: name 'graph_labels' is not defined

In [2]:
# from tensorflow.python.client import device_lib 
# print(device_lib.list_local_devices())
# !nvidia-smi
# import tensorflow as tf
# print(tf.__version__)
# tf.test.is_gpu_available()
# tf.compat.v1.Session()
# !conda list
# tf.test.is_built_with_cuda()
# tf.config.list_physical_devices('GPU')
# tf.test.is_gpu_available(
#     cuda_only=False, min_cuda_compute_capability=None
# )
# import warnings
# if not tf.test.gpu_device_name():
#     warnings.warn('No GPU found. Please ensure you have installed TensorFlow correctly')
# else:
#     print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

# !nvcc --version
# !nvidia-smi
# import tensorflow as tf

# sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))
# import os
# # os.environ["CUDA_VISIBLE_DEVICES"]="0,1,2"
# physical_devices = tf.config.experimental.list_physical_devices('XLA_GPU')
# print(physical_devices)
# # if physical_devices:
# #     tf.config.experimental.set_memory_growth(physical_devices[0], True)
# # tf.config.list_physical_devices(
# #     device_type=None)
# !cat /usr/local/cuda/include/cudnn.h | grep CUDNN_MAJOR -A 2

# with tf.device('device:XLA_GPU:0'):
#     a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')

# tf.config.experimental.list_physical_devices()
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import tensorflow as tf
print(tf.__version__)
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

ModuleNotFoundError: No module named 'tensorflow.compat'

In [21]:

##############################################################################################

In [22]:
    import pandas as pd
    import numpy as np

    import stellargraph as sg
    from stellargraph.mapper import PaddedGraphGenerator
    from stellargraph.layer import GCNSupervisedGraphClassification
    from stellargraph import StellarGraph

    from stellargraph import datasets
    import sklearn
    from sklearn import model_selection
    from sklearn.metrics import balanced_accuracy_score
    from IPython.display import display, HTML

    from tensorflow.keras import Model
    from tensorflow.keras.optimizers import Adam
    from tensorflow.keras.layers import Dense
    from tensorflow.keras.losses import binary_crossentropy
    from tensorflow.keras.callbacks import EarlyStopping
    import tensorflow.keras.metrics
    import tensorflow as tf
    import matplotlib.pyplot as plt

In [23]:
generator = PaddedGraphGenerator(graphs=exp1)

In [30]:
def create_graph_classification_model(generator):
    gc_model = GCNSupervisedGraphClassification(
        layer_sizes=[128, 128, 128,128],
        activations=["relu", "relu","relu", "relu"],
        generator=generator,
        dropout=0.5,
    )
    x_inp, x_out = gc_model.in_out_tensors()
    predictions = Dense(units=256, activation="relu")(x_out)
    predictions = Dense(units=128, activation="relu")(predictions)
    
    predictions = Dense(units=64, activation="relu")(predictions)
    
    predictions = Dense(units=32, activation="relu")(predictions)
    
    predictions = Dense(units=16, activation="relu")(predictions)
    predictions = Dense(units=1, activation="sigmoid")(predictions)

    # Let's create the Keras model and prepare it for training
    model = Model(inputs=x_inp, outputs=predictions)
#     model.compile(optimizer=Adam(0.005), loss=binary_crossentropy, metrics=["acc"]) 
    model.compile(optimizer=Adam(0.005), loss=binary_crossentropy, metrics=["acc",tf.keras.metrics.AUC()])

    return model

In [31]:
epochs = 200  # maximum number of training epochs
folds = 32 # the number of folds for k-fold cross validation
n_repeats = 5  # the number of repeats for repeated k-fold cross validation

In [32]:
es = EarlyStopping(
    monitor="val_loss", min_delta=0, patience=25, restore_best_weights=True
)

In [33]:
def train_fold(model, train_gen, test_gen, es, epochs):
    history = model.fit(
        train_gen, epochs=epochs, validation_data=test_gen, verbose=0, callbacks=[es],
    )
    # calculate performance on the test data and return along with history
    
    test_metrics = model.evaluate(test_gen, verbose=0)
#     print(model.metrics_names)
    test_acc = test_metrics[model.metrics_names.index("acc")]
#     test_bacc = test_metrics[3]
    test_auc = test_metrics[2]
    

    return history, test_acc, test_auc

In [34]:
def get_generators(train_index, test_index, graph_labels, batch_size):
    train_gen = generator.flow(
        train_index, targets=graph_labels.iloc[train_index].values, batch_size=batch_size
    )
    test_gen = generator.flow(
        test_index, targets=graph_labels.iloc[test_index].values, batch_size=batch_size
    )

    return train_gen, test_gen

In [36]:
test_accs = []
# test_raccs = []
# test_baccs = []
test_aucs = []
start = time.time()


stratified_folds = model_selection.RepeatedStratifiedKFold(
    n_splits=folds, n_repeats=n_repeats
).split(graph_labels, graph_labels)

for i, (train_index, test_index) in enumerate(stratified_folds):
    print(f"Training and evaluating on fold {i+1} out of {folds * n_repeats}...")
    train_gen, test_gen = get_generators(
        train_index, test_index, graph_labels, batch_size=30
    )

    model = create_graph_classification_model(generator)
    history, acc, auc = train_fold(model, train_gen, test_gen, es, epochs)
    
#     history, acc = train_fold(model, train_gen, test_gen, es, epochs)

    test_accs.append(acc)
#     test_raccs.append(racc)
    test_aucs.append(auc)

end = time.time()
print(end - start)

Training and evaluating on fold 1 out of 160...


TypeError: Failed to convert object of type <class 'list'> to Tensor. Contents: [-1, None, 1]. Consider casting elements to a supported type.

In [ ]:
print(
    f"Accuracy over all folds mean: {np.mean(test_accs)*100:.3}% and std: {np.std(test_accs)*100:.2}% and auc: {np.mean(test_aucs)}"
)

In [ ]:
plt.figure(figsize=(8, 6))
plt.hist(test_accs)
plt.xlabel("Accuracy")
plt.ylabel("Count")